In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
df4
# Assuming your DataFrame and target variable setup
x4 = df4.drop('Class', axis=1)
y4 = df4['Class']

X_train, X_test, y_train, y_test = train_test_split(x4, y4, test_size=0.4, random_state=0)


lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
predicted1 = lr.predict(X_test)

#r1 = (classification_report(y_test, predicted1,output_dict=True))
r1 = (classification_report(y_test, predicted1))
print(r1)


resampling = SMOTE()
lr = LogisticRegression(max_iter=1000)

pipeline = Pipeline([('SMOTE', resampling), ('Logistic Regression', lr)])
pipeline.fit(X_train, y_train)
predicted2 = pipeline.predict(X_test)

r2 = print(classification_report(y_test, predicted2))

resampling = BorderlineSMOTE(kind='borderline-1')
lr = LogisticRegression(max_iter=1000)

pipeline = Pipeline([('BSMOTE', resampling), ('Logistic Regression', lr)])
pipeline.fit(X_train, y_train)
predicted3 = pipeline.predict(X_test)

r3 = (classification_report(y_test, predicted3))
print(r3)

###HOW TO ACCESS THE CLASSIFICATION REPORT SCORES
r1_f1_score = round((classification_report(y_test, predicted1,output_dict=True))['macro avg']['f1-score'],2)
r2_f1_score = round((classification_report(y_test, predicted2,output_dict=True))['macro avg']['f1-score'],2)
r3_f1_score = round((classification_report(y_test, predicted3,output_dict=True))['macro avg']['f1-score'],2)

print(f'r1_f1_score:{r1_f1_score}\nr2_f1_score:{r2_f1_score}\nr3_f1_score:{r3_f1_score}')